Raphaël Chalicarne
28/2/18
PE 35

Dans ce Notebook, j'ai testé deux méthodes pour déterminer l'appartenance d'un point à la zone des Résidences, comprenant Comparat et Adoma.
- La méthode du "Rectangle"
- La méthode de "Pénalisation"

# Méthode Rectangle

On récupère les coordonnées GPS de quatre points A,B,C et D pour définir un polygone. On veut coder une fonction qui détermine si un point est dans ce polygone ou non.

<img src="residence.png"/>

In [7]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import time as t
import numpy as np

#Lecture du fichier
file = "Historique des positions raphael 170218.json"
df = pd.read_json(file)
donnee=df.get("locations")

#Récupération des latitudes, longitudes et des dates
lat=[]
long=[]
temps=[]

for i in range(len(donnee)):
    lat.append(donnee[i]["latitudeE7"]/1e7)
    long.append(donnee[i]["longitudeE7"]/1e7)
    temps.append(donnee[i]["timestampMs"])

def conv(temps):
    """Ce programme permet de convertir la liste des temps (de type string) en une liste des temps de type float, également utilisable par le module time"""
    n = len(temps)
    liste = [0]*n
    for i in range(n):
        t0 = 0.001*float(temps[i])
        liste[i]=t0
    return liste

temps = conv(temps)

Récupération des données GPS de OpenStreetMap
(au format [latitude, longitude])

In [8]:
A = [45.7834,4.77088]
B = [45.78346,4.77157]
C = [45.78122,4.7703]
D = [45.78134,4.76966]

Récupération des données GPS de GoogleMaps (les mesures sont plus précises)

In [9]:
X = [45.78253,4.77057] #Point dans le batiment X
S = [45.78321,4.76890] #Point sur le stade
ES = [45.782626, 4.770143] #Extrémité stade

#Mesures Google Maps
A = [45.783566, 4.770862]
B = [45.783438, 4.771527]
C = [45.781155, 4.770218]
D = [45.781364, 4.769370]

On définit un programme donnant le coefficient directeur a et l'ordonnée à l'origine b d'une fonction affine de la forme y = a*x + b, à partir de deux points appartenant à la droite.

In [11]:
def param_affine(pointA,pointB):
    """pointA = [x1,y1]
    pointB = [x2,y2]"""
    x1,y1 = pointA
    x2,y2 = pointB
    a = (y2-y1)/(x2-x1)
    b = (x2*y1-x1*y2)/(x2-x1)
    return a,b

On récupère les coordonnées des droites C1, C2, C3 et C4.

In [12]:
C1 = param_affine(A,B)
C2 = param_affine(B,C)
C3 = param_affine(C,D)
C4 = param_affine(D,A)

a1, b1 = C1
a2, b2 = C2
a3, b3 = C3
a4, b4 = C4

On définit alors la fonction is_in_Residence(point) qui renvoie un booléen selon que le point considéré soit dans le polygone ABCD ou non.

In [13]:
def is_in_Residence(point):
    """Entrée : point =[x,y]"""
    x, y = point
    yC1 = a1*x + b1
    yC2 = a2*x + b2
    yC3 = a3*x + b3
    yC4 = a4*x + b4
    if yC1>y>yC3 and yC4>y>yC2:
        return True
    else:
        return False

Ce programme ne fonctionne pas à cause d'incertitudes sur les coordonnées GPS de chaque point. En effet, en calculant les coefficients directeurs, on effectue une division par des coordonnées très proches, ce qui cause une grosse incertitude.

In [14]:
print("Le point X est dans la zone des résidences :",is_in_Residence(X))
print("Le point S est sur le stade, en dehors de la zone des résidences :",is_in_Residence(S))

Le point X est dans la zone des résidences : False
Le point S est sur le stade, en dehors de la zone des résidences : False


In [18]:
yC1 = a1*X[0] + b1
yC2 = a2*X[0] + b2
yC3 = a3*X[0] + b3
yC4 = a4*X[0] + b4
print("On devrait avoir",yC1," < ",X[1]," < ",yC3, "et",yC4," < ",X[1]," < ",yC2)

On devrait avoir 4.776244343647022  <  4.77057  <  4.764639052544197 et 4.7701600417806524  <  4.77057  <  4.771006381509515


# Méthode Pénalisation

Dans cette méthode, on mesure la distance d'un point à un segment caractéristique de la zone des résidences, pour déterminer si ce point est dans la zone ou non.

<img src="residence2.png"/>

Coordonnées des points A et B et détermination de l'équation caractéristique de la droite (AB).

In [19]:
A = [45.783316, 4.771082]
B = [45.781445, 4.769982]
a,b = param_affine(A,B)
c = -1/a

On propose alors une nouvelle fonction qui à partir d'une distance r0 considère que le point fait ou non partie de la zone des résidences.
Cette fonction est à améliorer pour prendre en compte les extrémités de la zone des résidences.

In [20]:
def is_in_Residence2(point):
    """point = [x,y]"""
    x,y = point
    d = y-c*x
    x0 = (d-b)/(a-c)
    y0 = a*x0+b
    r=np.sqrt((x-x0)**2+(y-y0)**2)
    r0 = 0.00045976248075233942
    if r<r0:
        return True
    else:
        return False

La valeur r0 a été déterminée empiriquement en calculant la distance du point "ES" à la droite caractéristique. 

In [22]:
print("S ne fait pas partie de la zone Résidences : ",is_in_Residence2(S))
print("X fait partie de la zone Résidences : ",is_in_Residence2(X))

S ne fait pas partie de la zone Résidences :  False
X fait partie de la zone Résidences :  True
